In [1]:
import pandas as pd
import requests
import json
import csv
import time
import datetime
from datetime import date

In [13]:
# date
after_start_date = date.fromisoformat('2016-04-01')
before_end_date = date.fromisoformat('2019-04-30')
after_start_seconds = (after_start_date -  date.fromisoformat('1970-01-01')).total_seconds()
before_end_seconds = (before_end_date -  date.fromisoformat('1970-01-01')).total_seconds()

print('after_start_seconds: ', after_start_seconds)
print('before_end_seconds: ', before_end_seconds)

after_start_seconds:  1459468800.0
before_end_seconds:  1556582400.0


# Retrieve the submissions

In [3]:
def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
          str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)+'&sort=asc'
#     url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
#           str(query)+'&size=1000'+'&subreddit='+str(sub)
#     print(url)
    r = requests.get(url)
#     print(r)
    data = json.loads(r.text)
    return data['data']

def getPushshiftData_noSubreddit(query, after, before):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
          str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&sort=asc'
#     url = 'https://api.pushshift.io/reddit/search/submission/?title='+\
#           str(query)+'&size=1000'+'&subreddit='+str(sub)
#     print(url)
    r = requests.get(url)
    print(r)
    data = json.loads(r.text)
    return data['data']

In [4]:
def collectSubData(subm):
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    try:
        subreddit = subm['subreddit']
    except KeyError:
        subreddit = "None"
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']
    
    subData.append((sub_id,title,url,author,score,subreddit, created,numComms,permalink,flair))
    subStats[sub_id] = subData

In [11]:
# keywords list
# keywords = ['vape', 'e liquid', 'e juice', 'ejuice', 'electronic cigarette', 'eliquid', 'e cig', 'ecig', 'e hookah', 'electronic cig', 'ehookah']
keywords = ['abcd']

### No specific subreddit

In [12]:
# query = "electronic-cigarette"
# sub = "electronic_cigarette"

for query in keywords:
    print('--------------------------------------')
    print('Query: ', query)
    after = str(after_start_seconds).split('.')[0]
    before = str(before_end_seconds).split('.')[0]

    subStats = {}
    subreddit_dict = {}

    data = getPushshiftData_noSubreddit(query, after, before)
    # Will run until all posts have been gathered 
    # from the 'after' date up until before date
    while len(data) > 0:
        for submission in data:
#             print(submission)
#             print(submission['title'])
#             print(submission['subreddit'])
#             print('-----------')
            collectSubData(submission)
    #         print(submission)
            try:
                subreddit = submission['subreddit']
            except KeyError:
                subreddit = "None"
            subreddit_dict[subreddit] = subreddit_dict.get(subreddit, 0) + 1
    #         break

        # Prepare for next iteration
        # print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
        after = data[-1]['created_utc']
        data = getPushshiftData_noSubreddit(query, after, before)

#     print(len(data))
    print('length of subStats.keys(): ', len(subStats.keys()))
#     print('subreddit_dict: ', subreddit_dict)
    print('Sorted: ', sorted(subreddit_dict.items(), key=lambda d: d[1], reverse=True))
    break
#     print('Sorted: ', sorted(subreddit_dict.items(), key=lambda d: d[1], reverse=True))


--------------------------------------
Query:  abcd
<Response [200]>
<Response [200]>
length of subStats.keys():  126
Sorted:  [('ABCDesis', 9), ('newsbotMARKET', 5), ('kidsvideo', 5), ('battlebots', 5), ('bakchodi', 3), ('u_puckhukids', 3), ('youtubehaiku', 3), ('rupaulsdragrace', 3), ('DragRaceExchange', 2), ('u_littlebabytube', 2), ('comsgntest', 2), ('funny', 2), ('anything', 2), ('explainlikeimfive', 2), ('test', 2), ('videos', 2), ('Blackops4', 2), ('HomeworkHelp', 2), ('business', 2), ('SummonSign', 2), ('cdcoverdesign', 1), ('IndieMusicPromotion', 1), ('u_Coinfans', 1), ('DiepioPartyLinks', 1), ('deals', 1), ('u_maxpro30', 1), ('ABCDesiSupportGroup', 1), ('Showerthoughts', 1), ('Dance', 1), ('dancedubbing', 1), ('freemusicpromo', 1), ('u_javbiz', 1), ('NoStupidQuestions', 1), ('bollywoodsongs', 1), ('pakistan', 1), ('india', 1), ('u_a1627', 1), ('u_vea8Ogh7xax_pdf', 1), ('u_ohW3see2uph_freepdf', 1), ('DeepFriedMemes', 1), ('u_TheeS3El2i_PDF', 1), ('u_eif6QueeBie_Book', 1), ('u_

### Specific subreddit

In [16]:
query = "san francisco"
sub = "electronic_cigarette"
after = str(after_start_seconds).split('.')[0]
before = str(before_end_seconds).split('.')[0]


subCount = 0
subStats = {}

data = getPushshiftData(query, after, before, sub)
# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:      
        collectSubData(submission)
#         print(submission)
#         subCount+=1
#         break
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftData(query, after, before, sub)
#     break
    
print(len(data))

50
2019-04-19 07:46:13
0


In [9]:
print('length of subStats.keys(): ', len(subStats.keys()))

length of subStats.keys():  5457


In [11]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

5457 submissions have added to list
1st entry is:
Need of advice for a begginer ( Smok AL85 and Zeus mono coil) created: electronic_cigarette
Last entry is:
Customer experience with MYLÉ created: electronic_cigarette


In [112]:
def updateSubs_file():
    upload_count = 0
    location = ""
    print("input filename of submission file, please add .csv")
    filename = input()
    file = location + filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")
        
updateSubs_file()

input filename of submission file, please add .csv
test.csv
1000 submissions have been uploaded


# Based on the submission, retrieve the comments

In [125]:
get_comments_id_from_submission_url = "https://api.pushshift.io/reddit/submission/comment_ids/"
get_comment_by_id_url = "https://api.pushshift.io/reddit/comment/search?ids="

In [131]:
# get comment_ids based on submission_id
for submission_id in subStats.keys():
    tmp_comments_list = []
    
    r = requests.get(get_comments_id_from_submission_url + submission_id)
    data = json.loads(r.text)
    comments_id_list = data['data']
    print(comments_id_list)
    
    # get comments based on comment_id
    for comment_id in comments_id_list:
        r_comment = requests.get(get_comment_by_id_url + comment_id)
        data_comment = json.loads(r_comment.text)
        comment = data_comment['data']
        print('comment: ', comment)
        
    break
    

['ds0npl7', 'ds0nrb1', 'ds0ridc', 'ds0rmkn', 'ds0tfjt', 'ds0twmr', 'ds0uado', 'ds0v8qw', 'ds0vmti', 'ds0vndj', 'ds0yzmw', 'ds0z0kc', 'ds0zftx', 'ds19414', 'ds2om6u']
comment:  [{'approved_at_utc': None, 'author': 'kwajr', 'author_flair_css_class': 'current', 'author_flair_text': 'Therion DNA 75c bf w/dead rabbit', 'banned_at_utc': None, 'body': 'Perhaps they did mess up the nic.', 'can_mod_post': False, 'collapsed': False, 'collapsed_reason': None, 'created_utc': 1514765577, 'distinguished': None, 'edited': False, 'id': 'ds0npl7', 'is_submitter': False, 'link_id': 't3_7nbz39', 'parent_id': 't3_7nbz39', 'permalink': '/r/electronic_cigarette/comments/7nbz39/brand_new_coil_brand_new_juice_way_too_harsh_and/ds0npl7/', 'retrieved_on': 1514765579, 'score': 1, 'stickied': False, 'subreddit': 'electronic_cigarette', 'subreddit_id': 't5_2qmlu'}]
comment:  [{'approved_at_utc': None, 'author': 'bennyblanco2121', 'author_flair_css_class': None, 'author_flair_text': None, 'banned_at_utc': None, 'bo

comment:  [{'approved_at_utc': None, 'author': 'jayden8250', 'author_flair_css_class': None, 'author_flair_text': None, 'banned_at_utc': None, 'body': 'great advice. definitely going to buy online next time. smoothest e juice?', 'can_mod_post': False, 'collapsed': False, 'collapsed_reason': None, 'created_utc': 1514782663, 'distinguished': None, 'edited': False, 'id': 'ds0z0kc', 'is_submitter': True, 'link_id': 't3_7nbz39', 'parent_id': 't1_ds0yzmw', 'permalink': '/r/electronic_cigarette/comments/7nbz39/brand_new_coil_brand_new_juice_way_too_harsh_and/ds0z0kc/', 'retrieved_on': 1514782664, 'score': 1, 'stickied': False, 'subreddit': 'electronic_cigarette', 'subreddit_id': 't5_2qmlu'}]
comment:  [{'approved_at_utc': None, 'author': 'prunmas', 'author_flair_css_class': None, 'author_flair_text': None, 'banned_at_utc': None, 'body': 'Almost anything that is either max VG or 80VG 20PG I buy from cloud compass but im Canadian I’m sure he ships to the US as well, his juice is very smooth and